<a href="https://colab.research.google.com/github/secutron/TesTime/blob/main/FixMatch_ignite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install requirements:

In [ ]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'


try:
  import torch_xla
  import ignite
except ImportError:
  VERSION = "nightly"
  # VERSION = "20200607"
  !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
  !python pytorch-xla-env-setup.py --version $VERSION
  !pip install --upgrade git+https://github.com/pytorch/ignite
  !pip install --upgrade --pre hydra-core
  !pip install wandb
  !pip uninstall -y pillow && CC="cc -mavx2" pip install --no-cache-dir --force-reinstall pillow-simd


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4264  100  4264    0     0  47377      0 --:--:-- --:--:-- --:--:-- 47377
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
Uninstalling torch-1.5.0+cu101:
Done updating TPU runtime: <Response [200]>
  Successfully uninstalled torch-1.5.0+cu101
Uninstalling torchvision-0.6.0+cu101:
  Successfully uninstalled torchvision-0.6.0+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 89.6 MiB/ 89.6 MiB]                                                
Operation completed over 1 objects/89.6 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly-cp36-cp36m-linux_x86_64.whl...
- [1 files][121.0 MiB/121.0 MiB]                                                
Operation completed over 1 objects/121.0 MiB.         

### Download dataset:

In [ ]:
# Download dataset:
from torchvision.datasets import CIFAR10

CIFAR10("/tmp/cifar10", train=True, download=True);

Extracting /tmp/cifar10/cifar-10-python.tar.gz to /tmp/cifar10


### Get the code:

In [ ]:
!git clone https://github.com/vfdev-5/FixMatch-pytorch/

#### Optionally, login to `W&B`

To skip logging to `W&B`, please set `online_exp_tracking.wandb=false` 

In [ ]:
# !wandb login <token>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


### Let's train ResNet18 model in a faster mode

In [ ]:
!cd FixMatch-pytorch && export PYTHONPATH=$PWD:$PYTHONPATH && \
  python main_fixmatch.py distributed.backend=xla-tpu distributed.nproc_per_node=8 online_exp_tracking.wandb=true solver.num_epochs=50 \
    ssl.confidence_threshold=0.7 ema_decay=0.9 ssl.cta_update_every=15 solver.optimizer.params.lr=0.1

2020-06-15 14:45:26,154 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'xla-tpu'
2020-06-15 14:45:26,154 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 8
	nnodes: 1
	node_rank: 0
2020-06-15 14:45:26,154 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7f7b29cdbd90>' in 8 processes
2020-06-15 14:46:42,829 FixMatch Training INFO: name: fixmatch
seed: 543
debug: false
model: resnet18
num_classes: 10
ema_decay: 0.9
solver:
  unsupervised_criterion:
    cls: torch.nn.CrossEntropyLoss
    params:
      reduction: none
  num_epochs: 50
  epoch_length: 128
  checkpoint_every: 500
  validate_every: 1
  resume_from: null
  optimizer:
    cls: torch.optim.SGD
    params:
      lr: 0.1
      momentum: 0.9
      weight_decay: 0.0001
      nesterov: false
  supervised_criterion:
    cls: torch.nn.CrossEntropyLoss
  lr_scheduler:
    cls: torch.optim.lr_scheduler.CosineAnnealing

### Let's train WRN-28-2 model

In [ ]:
!cd FixMatch-pytorch && export PYTHONPATH=$PWD:$PYTHONPATH && \
  python -u main_fixmatch.py model=WRN-28-2 distributed.backend=xla-tpu distributed.nproc_per_node=8 online_exp_tracking.wandb=true